In [ ]:
%matplotlib widget
from astrotime.loaders.synthetic import SyntheticElementLoader
from astrotime.plot.analysis import RawDatasetPlot
from astrotime.loaders.base import Loader, RDict
from astrotime.encoders.wavelet import WaveletAnalysisLayer, embedding_space
from astrotime.plot.analysis import EvaluatorPlot
from astrotime.trainers.model_evaluator import ModelEvaluator
from astrotime.config.context import astrotime_initialize
from astrotime.plot.base import SignalPlotFigure
from astrotime.models.cnn.cnn_baseline import get_model_from_cfg, ExpLoss, ExpHLoss, ExpU
import torch
from torch import nn
from hydra import initialize, compose

In [ ]:
version = "synthetic_period"
overrides = [ 'platform.gpu=-1', 'data.batch_size=1' ]
initialize(version_base=None, config_path="../config" )
cfg = compose( config_name=version, overrides=overrides )
device: torch.device = astrotime_initialize(cfg,version+".plot")

In [ ]:
device: torch.device = astrotime_initialize( cfg, version )
embedding_space_array, embedding_space_tensor = embedding_space(cfg.transform, device)

data_loader = SyntheticElementLoader(cfg.data)
embedding = WaveletAnalysisLayer( 'analysis', cfg.transform, embedding_space_tensor, device )
model: nn.Module = get_model_from_cfg( cfg.model, device, embedding, ExpU(cfg.data) )

evaluator = ModelEvaluator( device, data_loader, model, ExpHLoss(cfg.data) )
wplot = EvaluatorPlot("WWAnalysis Transform", evaluator )

fig = SignalPlotFigure([dplot,wplot])
fig.show()